# Read files

In [1]:
%%time

from antifraud import read_adj_dict_from_file, read_into_list_of_tuples
from graph_algorithms import Graph

# set file names
batch  = '../paymo_input/batch_payment.txt'
stream = '../paymo_input/stream_payment.txt'

# Read files
# Turn on verboisity to see the lines that did not conform to the csv format (Note: 
# it's the same 5 or so lines over and over again)
batch0 = read_adj_dict_from_file (batch, verbose = False)
stream = read_into_list_of_tuples (stream, verbose = False) 

# attach
batch = Graph (batch0.copy ())

CPU times: user 13.4 s, sys: 366 ms, total: 13.7 s
Wall time: 13.8 s


In [5]:
from graph_algorithms import next_order_adj_list

In [6]:
%%time

batch.adj_2 = next_order_adj_list (batch.adj_1)

CPU times: user 21.8 s, sys: 1.12 s, total: 22.9 s
Wall time: 22.9 s


In [7]:
%%time
batch.is_self_consistent ()

CPU times: user 47.4 s, sys: 4.1 ms, total: 47.4 s
Wall time: 47.5 s


True

In [ ]:
%%time
batch.adj_3 = next_order_adj_list (batch.adj_2)

In [ ]:
%%time
batch.is_self_consistent ()

In [2]:
feature = batch.copy ()
feature is batch

False

In [4]:
feature.is_self_consistent ()

True

# Features

In [ ]:
for k in range (10):
    print (stream [k], 'distance: ', batch.distance (*stream [k], 4))

In [ ]:
%%time
dist = [-1 for i in range (len (stream))]

for k in range (len (stream)):
    dist [k] = batch.distance (*stream [k], 4)

In [ ]:
def msg_if_ltn (graph, edge, n, message, add_edge = False):
    """
    
    """
    # check if the edge = (a, b) 
    ltn = graph.distance_lt_n (*edge, n)
    if add_edge and not ltn:
        graph.add_edge (edge)
    return message [0] if ltn else message [1]

## Feature 1
When anyone makes a payment to another user, they'll be notified if they've never made a transaction with that user before.

"unverified: You've never had a transaction with this user before. Are you sure you would like to proceed with this payment?"

In [ ]:
%%time

feature = Graph (batch0.copy ())

msg_true = "verified"
msg_false = "unverified: You've never had a transaction with this user before. Are you sure you would like to proceed with this payment?"
end = '\n'
message = [msg_true + end,
           msg_false + end]
output_file = 'output1.txt'
n = 1

with open (output_file, 'w') as output:
    for edge in stream:
        output.write (msg_if_ltn (feature, edge, n, message, add_edge = True))
print ('number of new nodes: {}\n'.format (feature.length - batch.length))

In [ ]:
feature.length

## Feature 2

The PayMo team is concerned that these warnings could be annoying because there are many users who haven't had transactions, but are still in similar social networks. To account for this, PayMo would like you to also implement this feature. When users make a payment, they'll be notified when the other user is outside of their "2nd-degree network".

"unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?"

In [ ]:
feature = Graph (batch0)
feature.length

In [ ]:
%%time

output_file = 'output2.txt'
n = 2

msg_true = "verified"
msg_false = 'unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?'
end = '\n'
message = [msg_true + end,
           msg_false + end]


feature = Graph (batch0.copy ())

with open (output_file, 'w') as output:
    for edge in stream:
        output.write (msg_if_ltn (feature, edge, n, message, add_edge = True))
#        if feature.distance_lt_n (*edge, n):
#            output.write (msg_true + end)
#        else:
#            output.write (msg_false + end)
#            feature.add_edge (edge)
print ('number of new nodes: {}\n'.format (feature.length - batch.length))

In [ ]:
feature.distance (42402, 42402)

In [ ]:
feature.length

## Feature 3

More generally, PayMo would like to extend this feature to larger social networks. Implement a feature to warn users only when they're outside the "4th degree friends network."

In [ ]:
%%time
msg_true = "verified"
msg_false = 'unverified: This user is not a friend or a "friend of a friend". Are you sure you would like to proceed with this payment?'
output_file = 'output3.txt'
n = 4

start_length = feature.length
with open (output_file, 'w') as output:
    for edge in stream:
        for edge in stream:
        if feature.distance_lt_n (*edge, n):
            output.write (msg_true)
        else:
            output.write (msg_false)
            feature.add_edge (edge)
            
print ('number of new nodes: {}\n'.format (feature.length - start_length))